In [ ]:
import sys
sys.path.append("../gaia_tools/")
sys.path.append("../scripts/")
import covariance_generation
import transformation_constants
import data_analysis
from mcmc_plots import *
import numpy as np
import emcee
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pickle
from pylab import *
from scipy.optimize import curve_fit
from cProfile import label
import pandas as pd

In [ ]:
dr3_path = '/local/mariacst/2022_v0_project/data/GaiaDR3_RV_RGB_fidelity.csv'
gaia_dr3 = pd.read_csv(dr3_path)
gaia_dr3.columns

In [ ]:
r_est_error = (gaia_dr3.B_rpgeo - gaia_dr3.b_rpgeo)/2
gaia_dr3['r_est_error'] = r_est_error

In [ ]:
C_icrs_new = covariance_generation.generate_covmat(gaia_dr3)
C_icrs_old = covariance_generation.generate_covmat(gaia_dr3, is_parallax=True)

In [ ]:
Z_0 = 25
R_0 = 8277

In [ ]:
data_array = gaia_dr3[["ra", "dec","r_est","pmra","pmdec","radial_velocity"]].to_numpy()
C_icrs_new_galcen = covariance_generation.transform_cov_matrix(C_icrs_new, data_array, "Cartesian", Z_0, R_0, is_bayes=True)
C_icrs_old_galcen = covariance_generation.transform_cov_matrix(C_icrs_old, data_array, "Cartesian", Z_0, R_0, is_bayes=True)

In [ ]:
def load_galactic_parameters():
   
   # Initial Galactocentric distance
   r_0 = 8277

   # Initial height over Galactic plane
   z_0 = 25

   # Initial solar vector
   v_sun = transformation_constants.V_SUN
   v_sun[0][0] = 11.1
   v_sun[1][0] = 251.5
   v_sun[2][0] = 8.59
   
   return r_0, z_0, v_sun
r_0, z_0, v_sun = load_galactic_parameters()

galcen_data = data_analysis.get_transformed_data(gaia_dr3,
                                        include_cylindrical = True,
                                        z_0 = z_0,
                                        r_0 = r_0,
                                        v_sun = v_sun,
                                        debug = True,
                                        is_bayes = True,
                                        is_source_included = True)

In [ ]:
data_array = galcen_data[["x", "y","r","phi","v_r","v_phi"]].to_numpy()
C_new_cylindrical = covariance_generation.transform_cov_matrix(C_icrs_new_galcen, data_array, "Cylindrical", Z_0, R_0)
C_old_cylindrical = covariance_generation.transform_cov_matrix(C_icrs_old_galcen, data_array, "Cylindrical", Z_0, R_0)

In [ ]:
C_new_cylindrical[:, 3, 3]
C_old_cylindrical[:, 4, 4]

In [ ]:
i = 3

fig = plt.figure(figsize=(8, 6))
h2 = plt.hist(np.sqrt(C_old_cylindrical[:, i, i]), 
            100, 
            range=(0, 100), 
            label='Old', 
            histtype='step', 
            linewidth=2)

h1 = plt.hist(C_new_cylindrical[:, i, i], 
            100, 
            range=(0, 50000), 
            label='Fixed', 
            histtype='step',
            linewidth=2)
plt.xlim(0, 50000)
plt.legend()
plt.yscale('log')

plt.xlabel(r'$\sigma_{Vr}$', fontdict={'fontsize' : 15})
plt.title(r'$\sigma_{Vr}$', fontsize = 18)